011 データを読み込んでみる

In [ ]:
import pandas as pd
uriage_data = pd.read_csv('./input/02/uriage.csv')
uriage_data.head()

In [ ]:
uriage_data.info()

`item_price`に欠損データ`NaN`のデータあることや`商`と`品`の間にスペースが入っているなど表記の揺れが大きいのが見て取れる  
欠損データがある列は`float64`と判断される

In [ ]:
print(pd.__version__)
kokyaku_data = pd.read_excel('./input/02/kokyaku_daicho.xlsx')
kokyaku_data.head()

In [ ]:
kokyaku_data.info()

欠損データはないが、スペースが入っていたり、登録日のフォーマットがまちまちだったりする

012 データの揺れを確認


In [ ]:
print(uriage_data['item_name'])

In [ ]:
print(uriage_data['item_price'])

013 データが欠損のまま集計

In [ ]:
uriage_data['purchase_date']
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
res

In [ ]:
res = uriage_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
res

014 商品名の揺れを補正する

In [ ]:
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data.item_name)))

戦略として、全角と半角スペースを除く、英字を半角大文字化すれば統一できそう

In [ ]:
uriage_data['item_name'] = uriage_data['item_name'].str.upper()
uriage_data['item_name'] = uriage_data['item_name'].str.replace('　', '')
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '')
print(pd.unique(uriage_data['item_name']))
print(len(pd.unique(uriage_data.item_name)))

商品名の揺れが補正されている

015 金額の欠損値を補完しよう

In [ ]:
uriage_data.isnull().any(axis=0)

`item_price`に欠損があることがわかる。一覧で見ると結構抜けている

In [ ]:
uriage_data['item_price']

In [ ]:
flg_is_null = uriage_data['item_price'].isnull()
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
    uriage_data.loc[(flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'] = price
    
uriage_data.isnull().any(axis=0)



最大値、最小値を確認してデータの補完状況を確認する

In [ ]:
for trg in list(uriage_data['item_name'].sort_values().unique()):
    print(trg + 'の最大値:' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].max()) + ' 最小値:' + str(uriage_data.loc[uriage_data['item_name']==trg]['item_price'].min(skipna=False)))


016: 顧客名の揺れを補正する

In [ ]:
kokyaku_data['顧客名'].head()

In [ ]:
uriage_data['customer_name'].head()

In [ ]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '')
kokyaku_data['顧客名'].head()

In [ ]:
print(kokyaku_data['顧客名'].unique())
print(uriage_data['customer_name'].unique())

017: 日付の揺れを補正する

顧客台帳はExcelで作成されており、人が入力しているため日付入力にも揺れがある

In [ ]:
print(uriage_data.columns)
print(kokyaku_data.columns)


In [ ]:
print(kokyaku_data['登録日'])

In [ ]:
flg_is_serial= kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial= kokyaku_data['登録日'].astype('str').str.isdigit()
print(flg_is_serial)

In [ ]:
from_serial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float64'), unit='D') + pd.to_datetime('1900/1/1')
from_serial

In [ ]:
from_string = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
from_string

In [ ]:
kokyaku_data['登録日'] = pd.concat([from_serial, from_string])
kokyaku_data

登録月の列を作成して集計を行う

In [ ]:
kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime("%Y%m")
result = kokyaku_data.groupby('登録年月').count()['顧客名']
result

In [ ]:
flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()
flg_is_serial.sum()

018 顧客名をキーにデータを結合する

In [ ]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')
join_data.drop('customer_name', axis=1)
join_data

019 クレンジングしたデータをダンプしておく

In [ ]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日']]
dump_data

In [ ]:
dump_data.to_csv('./input/02/dump_data.csv', index=False)

020 データを集計する

In [ ]:
import_data = pd.read_csv('./input/02/dump_data.csv')
import_data

In [ ]:
by_item = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='size', fill_value=0)
by_item

In [ ]:
by_price = import_data.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
by_price

In [ ]:
by_customer = import_data.pivot_table(index='purchase_month', columns='顧客名', values='item_price', aggfunc='size', fill_value=0)
by_customer

In [ ]:
by_region = import_data.pivot_table(index='purchase_month', columns='地域', values='item_price', aggfunc='size', fill_value=0)
by_region

期間内に購入のない顧客

In [ ]:
away_data = pd.merge(uriage_data, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')
print(away_data)
away_data[away_data.purchase_date.isnull()][['顧客名', 'メールアドレス', '登録日']]